In [1]:
import json
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import datetime
import helpers
import sheets
#import sf_2
import sf

In [2]:
import numpy as np
import re

In [3]:
#!pip install oauth2client

In [4]:
config_path = '/home/ruben.lopez/creds_sales_force.json'
#salesforce = sf_2.SF(config_path)
salesforce = sf.SF(config_path)
print("Getting User Data...")
user_data = salesforce.get_user_data()

Getting User Data...


In [5]:
sub_user_data = user_data[
    ['UserId', 'UserName', 'Email', 'Managers_Full_Name__c', 'RoleName', 'OnboardingStatus',
     'UserSubRegion']].reset_index(drop=True)
sub_user_data.rename(columns={'UserName': 'LeadOwnerName',
                              'Email': 'LeadOwnerEmail',
                              'Managers_Full_Name__c': 'LeadOwnerManager',
                              'RoleName': 'LeadOwnerRole',
                              'OnboardingStatus': 'LeadOwnerOnboardingStatus',
                              'UserSubRegion': 'LeadOwnerSubRegion'}, inplace=True)

# get latam data
latam_data = salesforce.get_region_data('LATAM')

Getting User Data ...
Getting Facility Data ...
Getting Leads Data ...
Getting Opportunity Data ...
Merging Global Data ...


/home/ruben.lopez/sales_opp and ex_heroku/helpers.py:765: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = opps_merged_data.append(leads_merged_data).reset_index(drop=True)


Global Data Pulled


In [6]:
latam_data

,LeadId,Company,Email,Phone,Website,LeadSource,LeadStatus,LeadOwnerId,LeadCreatedDate,Notes__c,...,ClosedWonDateWeek,ClosedWonDateMonth,ClosedLostDateWeek,ClosedLostDateMonth,NurturingDateWeek,NurturingDateMonth,ChurnDateWeek,ChurnDateMonth,LastActivityDateWeek,LastActivityDateMonth
0,00Q6T00001Ee0awUAB,Jou's Repostería- En desarrollo En 2-6 meses,yheredy20@outlook.com,+525525229787,None,CR - Inbound Website,Working,0055G000006QdMHQA0,2022-11-29 05:30:39,Estatus de proyecto: -En desarrollo\nCanal de ...,...,NaT,NaT,2021-09-06,2021-09-01,NaT,NaT,NaT,NaT,2021-08-16,2021-08-01
1,00Q6T00001DYZyJUAX,Havanna Alphaville Empresarial - Empresa Ativ...,havannadasa@gmail.com,+5511951968422,None,CR - Inbound Website,Qualified,0055G0000080PSnQAM,2022-09-10 21:40:31,- - CNPJ Ativo? - Horário de contato: Início d...,...,NaT,NaT,2022-09-19,2022-09-01,NaT,NaT,NaT,NaT,2022-09-12,2022-09-01
2,00Q6T00001Cm484UAB,La Dolce Doces Artesanais - Em desenvolvimento...,renata.ladolcecafe@gmail.com,+5511993666752,None,CR - Inbound Website,Qualified,0055G0000080PSnQAM,2022-08-09 11:43:40,"&#127856;Tortas, bolos e sobremesas - Não, ser...",...,2022-08-29,2022-08-01,NaT,NaT,NaT,NaT,NaT,NaT,2022-08-29,2022-08-01
3,00Q5G00000f8oAmUAI,Cookie Dough,marica.yasin@gmail.com,+19083671469,None,Inbound: Google Search,Qualified,0055G000006QsKuQAK,2022-05-31 00:01:50,Already Selling Through:\n- ✔️ Online / D2C (D...,...,NaT,NaT,2022-06-27,2022-06-01,NaT,NaT,NaT,NaT,2022-06-27,2022-06-01
4,00Q6T00001DYw63UAD,Oakland Fortune Factory,alex@oaklandfortunefactory.com,(510) 529-4531,http://www.oaklandfortunefactory.com/,CR - Outbound Data Science,Qualified,0055G000006QsKuQAK,2022-09-16 20:25:01,"Cookie shop,Bakery,Dessert shop,Gift shop,Tour...",...,NaT,NaT,2022-10-03,2022-10-01,NaT,NaT,NaT,NaT,2022-10-10,2022-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23402,00Q6T00001DYswOUAT,Las Empanadas - Empresa Ativa - Abre,luis.selva@uol.com.br,+5521979207999,None,CR - Inbound Website,Qualified,005f4000005Nv3GAAS,2022-09-15 23:50:11,- - CNPJ Ativo? - Horário de contato: Final da...,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
23403,00Q6T00001DZKkAUAX,Enchanted Sweet Shoppe,primalnotary@gmail.com,(714) 365-0612,None,CR - Outbound Data Science,Qualified,0056T000008oDmXQAU,2022-09-23 17:32:40,None,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
23404,00Q6T00001DBoaIUAT,Fairy Tale Bakers,fairytalebakers@yahoo.com,909-896-8740,None,CR - Outbound Data Science,Qualified,0056T000008oDmXQAU,2022-10-21 14:08:58,None,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
23405,00Q6T00001DD1AxUAL,Baby O’s Donuts,alexorfin@gmail.com,+17073865098,http://www.baby-os.com/index.html,Inbound: Website,Qualified,0055G000006QsKuQAK,2022-11-11 17:54:42,Already Selling Through:\n- ✔️ Online / D2C (D...,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [7]:
latam_data.columns

Index(['LeadId', 'Company', 'Email', 'Phone', 'Website', 'LeadSource',
       'LeadStatus', 'LeadOwnerId', 'LeadCreatedDate', 'Notes__c',
       ...
       'ClosedWonDateWeek', 'ClosedWonDateMonth', 'ClosedLostDateWeek',
       'ClosedLostDateMonth', 'NurturingDateWeek', 'NurturingDateMonth',
       'ChurnDateWeek', 'ChurnDateMonth', 'LastActivityDateWeek',
       'LastActivityDateMonth'],
      dtype='object', length=116)

In [8]:
# get status_changes
#salesforce = sf_2.SF(config_path)
salesforce = sf.SF(config_path)
lead_status_data = salesforce.get_region_lead_status_changes('LATAM')


latam_data['Unqualified_Reason__c'] = latam_data[['Lead_Closed_Reason__c',
                                                  'Unqualified_Reason__c']].apply(lambda x: helpers.
                                                                                  first_non_null(x[0], x[1]), axis=1)

latam_data.rename(columns={'Unqualified_Reason__c': 'Main_Closed_Reason'}, inplace=True)

# latam_data['LeadSource'] = np.where(~pd.isnull(latam_data.Source), latam_data.Source,
#                                    latam_data.LeadSource)

latam_data = latam_data[['LeadId', 'LeadOwnerId', 'LeadCreatedDate', 'Company', 'Email', 'Phone', 'LeadSource',
                         'LeadStatus', 'Main_Closed_Reason', 'Lead_Source_Detail__c', 'Opportunity_Stage__c',
                         'Estimated_Weekly_Orders__c', 'Refreshed_Lead__c', 'Number_of_Locations__c',
                         'IsWon', 'StageName', 'KitchenCity', 'KitchenCountry', 'UserCity', 'UserCountry',
                         'UserSubRegion', 'Kitchen_Type__c', 'FacilityName', 'QualifiedDate', 'AccountId',
                         'mrr', 'Negotiation_Date__c', 'Commitment_Date__c', 'Closed_Won_Date__c',
                         'General_Lost_Reason__c', 'Specific_Lost_Reason__c',
                         'TourCompleted__c', 'Final_Tour_Date__c', 'SDR_Attribution__c',
                         'SQL_Date__c', 'RecordTypeName', 'Facility_Tour_Date__c', 'UserRegion'
                         ,'UTM_Campaign__c', 'UTM_Content__c', 'UTM_Medium__c', 'UTM_Source__c', 'UTM_Term__c',
                                'utm_geo__c', 'UTM_Ad__c', 'UTM_Channel__c','Cuisine_Type__c']]

Getting Status Changes...
Status Changes Pulled...


In [9]:
latam_data

,LeadId,LeadOwnerId,LeadCreatedDate,Company,Email,Phone,LeadSource,LeadStatus,Main_Closed_Reason,Lead_Source_Detail__c,...,UserRegion,UTM_Campaign__c,UTM_Content__c,UTM_Medium__c,UTM_Source__c,UTM_Term__c,utm_geo__c,UTM_Ad__c,UTM_Channel__c,Cuisine_Type__c
0,00Q6T00001Ee0awUAB,0055G000006QdMHQA0,2022-11-29 05:30:39,Jou's Repostería- En desarrollo En 2-6 meses,yheredy20@outlook.com,+525525229787,CR - Inbound Website,Working,None,utm_source=smv&utm_channel=fb&utm_campaign=mex...,...,LATAM,mexico_smv_cdmx,None,None,smv,None,None,ex_cbo_post_buenfin3_vops_linkad_interests,fb,None
1,00Q6T00001DYZyJUAX,0055G0000080PSnQAM,2022-09-10 21:40:31,Havanna Alphaville Empresarial - Empresa Ativ...,havannadasa@gmail.com,+5511951968422,CR - Inbound Website,Qualified,None,utm_source=ig&utm_medium=paid&utm_channel=ig&u...,...,LATAM,CLOUDRETAIL_BRAZIL_SP_LA_TOFU,None,paid,ig,xxxxx,brazil_sp,STATIC_NOHIRINGEXPANDYELLOW-HEADLINE1-BODY1,ig,None
2,00Q6T00001Cm484UAB,0055G0000080PSnQAM,2022-08-09 11:43:40,La Dolce Doces Artesanais - Em desenvolvimento...,renata.ladolcecafe@gmail.com,+5511993666752,CR - Inbound Website,Qualified,None,utm_source=xxxxx&utm_medium=xxxxx&utm_channel=...,...,LATAM,xxxxx,None,xxxxx,xxxxx,xxxxx,xxxxx,xxxxx,xxxxx,CR - Baked Goods
3,00Q5G00000f8oAmUAI,0055G000006QsKuQAK,2022-05-31 00:01:50,Cookie Dough,marica.yasin@gmail.com,+19083671469,Inbound: Google Search,Qualified,Did Not Reach,None,...,US/CAN,nationwide_nationwide_branded,responsive,paid_search,google,Cloudretail,nationwide_nationwide,None,gsearch,None
4,00Q6T00001DYw63UAD,0055G000006QsKuQAK,2022-09-16 20:25:01,Oakland Fortune Factory,alex@oaklandfortunefactory.com,(510) 529-4531,CR - Outbound Data Science,Qualified,None,None,...,US/CAN,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23402,00Q6T00001DYswOUAT,005f4000005Nv3GAAS,2022-09-15 23:50:11,Las Empanadas - Empresa Ativa - Abre,luis.selva@uol.com.br,+5521979207999,CR - Inbound Website,Qualified,Did Not Reach,utm_source=ig&utm_medium=paid&utm_channel=ig&u...,...,LATAM,CLOUDRETAIL_BRAZIL_RIO_LA_TOFU,None,paid,ig,xxxxx,brazil_rio,VIDEO_TIKTOK-HEADLINE1-BODY1,ig,NaN
23403,00Q6T00001DZKkAUAX,0056T000008oDmXQAU,2022-09-23 17:32:40,Enchanted Sweet Shoppe,primalnotary@gmail.com,(714) 365-0612,CR - Outbound Data Science,Qualified,None,None,...,US/CAN,None,None,None,None,None,None,None,None,NaN
23404,00Q6T00001DBoaIUAT,0056T000008oDmXQAU,2022-10-21 14:08:58,Fairy Tale Bakers,fairytalebakers@yahoo.com,909-896-8740,CR - Outbound Data Science,Qualified,None,None,...,US/CAN,None,None,None,None,None,None,None,None,NaN
23405,00Q6T00001DD1AxUAL,0055G000006QsKuQAK,2022-11-11 17:54:42,Baby O’s Donuts,alexorfin@gmail.com,+17073865098,Inbound: Website,Qualified,None,None,...,US/CAN,xxxxx,xxxxx,xxxxx,None,xxxxx,xxxxx,None,None,NaN


In [10]:
user_data

,UserId,UserName,Email,Alias,Title,City,Country,Region__c,IsActive,Managers_Full_Name__c,...,SDR_Start_Date__c,SDR_End_Date__c,UserSubRegion,IsRamped,IsRamping,IsOnboarding,OnboardingStatus,ProfileName,RoleName,CreatedByName
0,005f4000002Z9jaAAC,Automated Process,rsandoval@00df4000003jvm8eai,autoproc,None,San Francisco,US,None,True,None,...,NaT,NaT,US,False,False,True,Onboarding,NaN,NaN,NaN
1,005f4000004jr7lAAA,System,automatedcase@00df4000003jvm8eai.ext,autocase,None,None,None,None,True,None,...,NaT,NaT,NaN,False,False,True,Onboarding,NaN,NaN,NaN
2,0055G0000081IlvQAE,kitchenonboarding Site Guest User,kitchenonboarding@cloudkitchens.force.com,guest,None,None,None,None,True,None,...,NaT,NaT,NaN,False,False,True,Onboarding,kitchenonboarding Profile,NaN,Septian Ekavandy
3,0056T000008nFkMQAU,Facility Site Site Guest User,facility_site@cssfacility.force.com,guest,None,None,None,None,True,None,...,NaT,NaT,NaN,False,False,True,Onboarding,Facility Site Profile,NaN,Afolabi Onanuga
4,0056T0000090yhlQAA,Launcher Site Guest User,launcher@cssfacility.force.com,guest,None,None,None,None,True,None,...,NaT,NaT,NaN,False,False,True,Onboarding,Launcher Profile,NaN,Lucas Santos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3334,005f4000006ZooLAAS,ryan test,ryan@delegate.team,rtest,None,None,None,None,False,None,...,NaT,NaT,NaN,False,False,True,Terminated,System Administrator,NaN,Ryan Coleman
3335,005f4000004p9VgAAI,Platform Integration User,noreply@salesforce.com,platint,None,None,None,None,True,None,...,NaT,NaT,NaN,False,False,True,Onboarding,NaN,NaN,NaN
3336,005f4000002Z9jdAAC,Chatter Expert,noreply@chatter.salesforce.com,Chatter,None,San Francisco,US,US/CAN,False,None,...,NaT,NaT,US,False,False,True,Terminated,Chatter Free User,NaN,Paco Robert
3337,005f4000004iTs4AAE,Ethan Ow,ethan.ow@cloudkitchens.com,eow,None,None,None,None,False,None,...,NaT,NaT,NaN,False,False,True,Terminated,Chatter External User,NaN,Domenico Tan


In [11]:
lead_status_data

,LeadId,WorkingDate,ProcessedDate
0,00Q5G00000Sfz0fUAB,2022-11-28 18:25:17,NaT
1,00Q5G00000Sg3KxUAJ,2022-11-07 19:02:12,NaT
2,00Q5G00000SgSJHUA3,2022-08-22 16:46:00,2022-08-23 19:40:56
3,00Q5G00000SgkFAUAZ,2022-07-30 08:16:48,NaT
4,00Q5G00000SgkFBUAZ,2022-07-30 07:30:17,NaT
...,...,...,...
26530,00Qf400000S7bGyEAJ,NaT,2022-09-06 21:44:37
26531,00Qf400000S7nQLEAZ,NaT,2022-07-12 22:54:22
26532,00Qf400000S8h2lEAB,NaT,2022-08-26 15:27:21
26533,00Qf400000SAuISEA1,NaT,2022-07-19 17:32:47


In [12]:
sub_user_data

,UserId,LeadOwnerName,LeadOwnerEmail,LeadOwnerManager,LeadOwnerRole,LeadOwnerOnboardingStatus,LeadOwnerSubRegion
0,005f4000002Z9jaAAC,Automated Process,rsandoval@00df4000003jvm8eai,None,NaN,Onboarding,US
1,005f4000004jr7lAAA,System,automatedcase@00df4000003jvm8eai.ext,None,NaN,Onboarding,NaN
2,0055G0000081IlvQAE,kitchenonboarding Site Guest User,kitchenonboarding@cloudkitchens.force.com,None,NaN,Onboarding,NaN
3,0056T000008nFkMQAU,Facility Site Site Guest User,facility_site@cssfacility.force.com,None,NaN,Onboarding,NaN
4,0056T0000090yhlQAA,Launcher Site Guest User,launcher@cssfacility.force.com,None,NaN,Onboarding,NaN
...,...,...,...,...,...,...,...
3334,005f4000006ZooLAAS,ryan test,ryan@delegate.team,None,NaN,Terminated,NaN
3335,005f4000004p9VgAAI,Platform Integration User,noreply@salesforce.com,None,NaN,Onboarding,NaN
3336,005f4000002Z9jdAAC,Chatter Expert,noreply@chatter.salesforce.com,None,NaN,Terminated,US
3337,005f4000004iTs4AAE,Ethan Ow,ethan.ow@cloudkitchens.com,None,NaN,Terminated,NaN


In [13]:
# ###################### PULL LAST WEEK ACTIVITY DATA #######################
user_ids = list(user_data[(user_data['Region__c'].isin(['LATAM','US/CAN'])) &
                          (user_data['RoleName'] == 'Account Executive')]['UserId'].unique())

# add owner data
latam_data = latam_data.merge(sub_user_data,
                              left_on="LeadOwnerId",
                              right_on="UserId",
                              how="left")

# add status changes
latam_data = latam_data.merge(
    lead_status_data,
    left_on="LeadId",
    right_on="LeadId",
    how="left")



In [14]:
latam_data

,LeadId,LeadOwnerId,LeadCreatedDate,Company,Email,Phone,LeadSource,LeadStatus,Main_Closed_Reason,Lead_Source_Detail__c,...,Cuisine_Type__c,UserId,LeadOwnerName,LeadOwnerEmail,LeadOwnerManager,LeadOwnerRole,LeadOwnerOnboardingStatus,LeadOwnerSubRegion,WorkingDate,ProcessedDate
0,00Q6T00001Ee0awUAB,0055G000006QdMHQA0,2022-11-29 05:30:39,Jou's Repostería- En desarrollo En 2-6 meses,yheredy20@outlook.com,+525525229787,CR - Inbound Website,Working,None,utm_source=smv&utm_channel=fb&utm_campaign=mex...,...,None,0055G000006QdMHQA0,Jader Navarro,jader.navarro@cloudkitchens.com,Irving Ramirez,Sales Development Representative CR,Terminated,Mexico,2022-11-29 15:31:22,NaT
1,00Q6T00001DYZyJUAX,0055G0000080PSnQAM,2022-09-10 21:40:31,Havanna Alphaville Empresarial - Empresa Ativ...,havannadasa@gmail.com,+5511951968422,CR - Inbound Website,Qualified,None,utm_source=ig&utm_medium=paid&utm_channel=ig&u...,...,None,0055G0000080PSnQAM,Bruno Galhota Dos Reis,bruno.reis@cloudkitchens.com,Laura Luiza,Sales Development Representative CR,Onboarding,Brazil-X,2022-09-12 13:53:44,2022-09-12 19:45:44
2,00Q6T00001Cm484UAB,0055G0000080PSnQAM,2022-08-09 11:43:40,La Dolce Doces Artesanais - Em desenvolvimento...,renata.ladolcecafe@gmail.com,+5511993666752,CR - Inbound Website,Qualified,None,utm_source=xxxxx&utm_medium=xxxxx&utm_channel=...,...,CR - Baked Goods,0055G0000080PSnQAM,Bruno Galhota Dos Reis,bruno.reis@cloudkitchens.com,Laura Luiza,Sales Development Representative CR,Onboarding,Brazil-X,2022-08-09 13:36:38,2022-08-09 19:57:13
3,00Q5G00000f8oAmUAI,0055G000006QsKuQAK,2022-05-31 00:01:50,Cookie Dough,marica.yasin@gmail.com,+19083671469,Inbound: Google Search,Qualified,Did Not Reach,None,...,None,0055G000006QsKuQAK,Chris Scarpa,christopher.scarpa@cloudkitchens.com,Mike McLaughlin,Sales Development Representative CR,Ramped,United States of America,NaT,NaT
4,00Q6T00001DYw63UAD,0055G000006QsKuQAK,2022-09-16 20:25:01,Oakland Fortune Factory,alex@oaklandfortunefactory.com,(510) 529-4531,CR - Outbound Data Science,Qualified,None,None,...,None,0055G000006QsKuQAK,Chris Scarpa,christopher.scarpa@cloudkitchens.com,Mike McLaughlin,Sales Development Representative CR,Ramped,United States of America,2022-09-22 20:53:57,2022-09-22 21:31:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23402,00Q6T00001DYswOUAT,005f4000005Nv3GAAS,2022-09-15 23:50:11,Las Empanadas - Empresa Ativa - Abre,luis.selva@uol.com.br,+5521979207999,CR - Inbound Website,Qualified,Did Not Reach,utm_source=ig&utm_medium=paid&utm_channel=ig&u...,...,NaN,005f4000005Nv3GAAS,Catalina Moncada,catalina.moncada@cloudkitchens.com,Leo Tang,Super Admin,Ramped,CCAC,2022-09-16 18:26:17,2022-10-03 14:48:26
23403,00Q6T00001DZKkAUAX,0056T000008oDmXQAU,2022-09-23 17:32:40,Enchanted Sweet Shoppe,primalnotary@gmail.com,(714) 365-0612,CR - Outbound Data Science,Qualified,None,None,...,NaN,0056T000008oDmXQAU,Bayne Hagood,bayne.hagood@cloudkitchens.com,Bryon Craven,Sales Development Representative CR,Ramping,United States of America,2022-09-29 20:52:55,2022-10-12 21:25:46
23404,00Q6T00001DBoaIUAT,0056T000008oDmXQAU,2022-10-21 14:08:58,Fairy Tale Bakers,fairytalebakers@yahoo.com,909-896-8740,CR - Outbound Data Science,Qualified,None,None,...,NaN,0056T000008oDmXQAU,Bayne Hagood,bayne.hagood@cloudkitchens.com,Bryon Craven,Sales Development Representative CR,Ramping,United States of America,2022-10-21 17:59:55,2022-10-31 20:49:42
23405,00Q6T00001DD1AxUAL,0055G000006QsKuQAK,2022-11-11 17:54:42,Baby O’s Donuts,alexorfin@gmail.com,+17073865098,Inbound: Website,Qualified,None,None,...,NaN,0055G000006QsKuQAK,Chris Scarpa,christopher.scarpa@cloudkitchens.com,Mike McLaughlin,Sales Development Representative CR,Ramped,United States of America,NaT,2022-11-15 00:16:29


In [15]:

latam_data['DaysToWorking'] = latam_data[['LeadCreatedDate', 'WorkingDate']].apply(
    lambda x: helpers.get_time_to_end_date(x[0], x[1])['Days'], axis=1)
latam_data['HoursToWorking'] = latam_data[['LeadCreatedDate', 'WorkingDate']].apply(
    lambda x: helpers.get_time_to_end_date(x[0], x[1])['Hours'], axis=1)

latam_data['DaysToProcessed'] = latam_data[['LeadCreatedDate', 'ProcessedDate']].apply(
    lambda x: helpers.get_time_to_end_date(x[0], x[1])['Days'], axis=1)
latam_data['HoursToProcessed'] = latam_data[['LeadCreatedDate', 'ProcessedDate']].apply(
    lambda x: helpers.get_time_to_end_date(x[0], x[1])['Hours'], axis=1)

latam_data['DaysToQualified'] = latam_data[['LeadCreatedDate', 'QualifiedDate']].apply(
    lambda x: helpers.get_time_to_end_date(x[0], x[1])['Days'], axis=1)

latam_data['DaysToNegotiating'] = latam_data[['LeadCreatedDate', 'Negotiation_Date__c']].apply(
    lambda x: helpers.get_time_to_end_date(x[0], x[1])['Days'], axis=1)

latam_data['DaysToClosedWon'] = latam_data[['LeadCreatedDate', 'Closed_Won_Date__c']].apply(
    lambda x: helpers.get_time_to_end_date(x[0], x[1])['Days'], axis=1)

latam_data['LeadCreatedDateWeek'] = latam_data['LeadCreatedDate'].apply(lambda x: helpers.get_week_from_date(x))
latam_data['LeadCreatedDateMonth'] = latam_data['LeadCreatedDate'].apply(lambda x: helpers.get_month_from_date(x))


latam_data['Final_Tour_Date_week'] = latam_data['Final_Tour_Date__c'].apply(lambda x: helpers.get_week_from_date(x))
latam_data['Final_Tour_Date_month'] = latam_data['Final_Tour_Date__c'].apply(lambda x: helpers.get_month_from_date(x))

latam_data['rec_day'] = np.where(pd.isnull(latam_data.Closed_Won_Date__c),
                          np.where(pd.isnull(latam_data.Final_Tour_Date__c),
                                   np.where(pd.isnull(latam_data.QualifiedDate),
                                            latam_data.LeadCreatedDate, latam_data.QualifiedDate),
                                   latam_data.Final_Tour_Date__c), latam_data.Closed_Won_Date__c)

latam_data.Facility_Tour_Date__c = pd.to_datetime(latam_data.Facility_Tour_Date__c)
latam_data.Facility_Tour_Date__c = latam_data.Facility_Tour_Date__c.dt.date

latam_data_no_filter = latam_data.copy()
latam_data = np.nan
print('Processing Retail Info')

Processing Retail Info


In [16]:
latam_data_no_filter.columns

Index(['LeadId', 'LeadOwnerId', 'LeadCreatedDate', 'Company', 'Email', 'Phone',
       'LeadSource', 'LeadStatus', 'Main_Closed_Reason',
       'Lead_Source_Detail__c', 'Opportunity_Stage__c',
       'Estimated_Weekly_Orders__c', 'Refreshed_Lead__c',
       'Number_of_Locations__c', 'IsWon', 'StageName', 'KitchenCity',
       'KitchenCountry', 'UserCity', 'UserCountry', 'UserSubRegion',
       'Kitchen_Type__c', 'FacilityName', 'QualifiedDate', 'AccountId', 'mrr',
       'Negotiation_Date__c', 'Commitment_Date__c', 'Closed_Won_Date__c',
       'General_Lost_Reason__c', 'Specific_Lost_Reason__c', 'TourCompleted__c',
       'Final_Tour_Date__c', 'SDR_Attribution__c', 'SQL_Date__c',
       'RecordTypeName', 'Facility_Tour_Date__c', 'UserRegion',
       'UTM_Campaign__c', 'UTM_Content__c', 'UTM_Medium__c', 'UTM_Source__c',
       'UTM_Term__c', 'utm_geo__c', 'UTM_Ad__c', 'UTM_Channel__c',
       'Cuisine_Type__c', 'UserId', 'LeadOwnerName', 'LeadOwnerEmail',
       'LeadOwnerManager', 'Lea

In [17]:
budget = latam_data_no_filter[['LeadId', 'LeadOwnerId', 'LeadCreatedDate', 'Company', 'Email',
                               'Phone', 'LeadSource', 'LeadStatus', 'Main_Closed_Reason', 'Lead_Source_Detail__c',
                               'Opportunity_Stage__c', 'Estimated_Weekly_Orders__c', 'Refreshed_Lead__c',
                               'Number_of_Locations__c', 'IsWon', 'StageName', 'KitchenCity', 'KitchenCountry',
                               'UserCity', 'UserCountry', 'UserSubRegion', 'Kitchen_Type__c', 'FacilityName',
                               'QualifiedDate', 'AccountId', 'mrr', 'Negotiation_Date__c',
                               'Commitment_Date__c', 'Closed_Won_Date__c', 'General_Lost_Reason__c',
                               'Specific_Lost_Reason__c', 'TourCompleted__c', 'Final_Tour_Date__c',
                               'Facility_Tour_Date__c', 'SDR_Attribution__c', 'LeadOwnerName', 'LeadOwnerEmail',
                               'SQL_Date__c',
                               'LeadOwnerRole', 'LeadOwnerOnboardingStatus', 'RecordTypeName', 'WorkingDate',
                               'ProcessedDate', 'DaysToWorking', 'HoursToWorking', 'DaysToProcessed',
                              'UTM_Campaign__c', 'UTM_Content__c', 'UTM_Medium__c', 'UTM_Source__c',
       'UTM_Term__c', 'utm_geo__c', 'UTM_Ad__c', 'UTM_Channel__c','Cuisine_Type__c']][latam_data_no_filter.UserRegion=='LATAM']


budget['rec_day'] = np.where(pd.isnull(budget.Closed_Won_Date__c),
                          np.where(pd.isnull(budget.Final_Tour_Date__c),
                                   np.where(pd.isnull(budget.QualifiedDate),
                                            budget.LeadCreatedDate, budget.QualifiedDate),
                                   budget.Final_Tour_Date__c), budget.Closed_Won_Date__c)

budget['CLSD'] = budget.Lead_Source_Detail__c.apply(helpers.clean_lead_source_detail)

In [18]:
# adding filter to 6 months
today = datetime.datetime.utcnow()
target = today - datetime.timedelta(days=180)
# budget = budget[lambda x: x['rec_day'].between(datetime.datetime(target.year, target.month, 1), today)]
print('Budget shape:', budget.shape)

budget['LeadCreatedDate'] = budget['LeadCreatedDate'].apply(lambda x: helpers.convert_date_to_str(x))
budget['QualifiedDate'] = budget['QualifiedDate'].apply(lambda x: helpers.convert_date_to_str(x))
# budget['Pitching_Date__c'] = budget['Pitching_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Negotiation_Date__c'] = budget['Negotiation_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Commitment_Date__c'] = budget['Commitment_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Closed_Won_Date__c'] = budget['Closed_Won_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Final_Tour_Date__c'] = budget['Final_Tour_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Facility_Tour_Date__c'] = budget['Facility_Tour_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['WorkingDate'] = budget['WorkingDate'].apply(lambda x: helpers.convert_date_to_str(x))
budget['ProcessedDate'] = budget['ProcessedDate'].apply(lambda x: helpers.convert_date_to_str(x))


Budget shape: (16249, 57)


In [19]:
# filter latam data
budget['is_inbound'] = budget['LeadSource'].apply(lambda x: helpers.check_is_inbound(x))




In [20]:
budget

,LeadId,LeadOwnerId,LeadCreatedDate,Company,Email,Phone,LeadSource,LeadStatus,Main_Closed_Reason,Lead_Source_Detail__c,...,UTM_Medium__c,UTM_Source__c,UTM_Term__c,utm_geo__c,UTM_Ad__c,UTM_Channel__c,Cuisine_Type__c,rec_day,CLSD,is_inbound
0,00Q6T00001Ee0awUAB,0055G000006QdMHQA0,11/29/2022,Jou's Repostería- En desarrollo En 2-6 meses,yheredy20@outlook.com,+525525229787,CR - Inbound Website,Working,None,utm_source=smv&utm_channel=fb&utm_campaign=mex...,...,None,smv,None,None,ex_cbo_post_buenfin3_vops_linkad_interests,fb,None,2021-08-13 00:00:00,utm_source=smv&utm_channel=fb&utm_campaign=mex...,True
1,00Q6T00001DYZyJUAX,0055G0000080PSnQAM,09/10/2022,Havanna Alphaville Empresarial - Empresa Ativ...,havannadasa@gmail.com,+5511951968422,CR - Inbound Website,Qualified,None,utm_source=ig&utm_medium=paid&utm_channel=ig&u...,...,paid,ig,xxxxx,brazil_sp,STATIC_NOHIRINGEXPANDYELLOW-HEADLINE1-BODY1,ig,None,2022-09-12 00:00:00,utm_source=ig&utm_channel=ig&utm_campaign=CLOU...,True
2,00Q6T00001Cm484UAB,0055G0000080PSnQAM,08/09/2022,La Dolce Doces Artesanais - Em desenvolvimento...,renata.ladolcecafe@gmail.com,+5511993666752,CR - Inbound Website,Qualified,None,utm_source=xxxxx&utm_medium=xxxxx&utm_channel=...,...,xxxxx,xxxxx,xxxxx,xxxxx,xxxxx,xxxxx,CR - Baked Goods,2022-08-29 00:00:00,utm_source=xxxxx&utm_channel=xxxxx&utm_campaig...,True
5,00Q6T00001DYa0ZUAT,0055G0000080PSnQAM,09/10/2022,Empório Bertoni - Em desenvolvimento - Abre em...,bertoniemporio@gmail.com,+5511993472058,CR - Inbound Website,Qualified,None,utm_source=xxxxx&utm_medium=xxxxx&utm_channel=...,...,xxxxx,xxxxx,xxxxx,xxxxx,xxxxx,xxxxx,None,2022-09-20 00:00:00,utm_source=xxxxx&utm_channel=xxxxx&utm_campaig...,True
6,00Q6T00001CkdArUAJ,0055G0000081NCHQA2,07/20/2022,Terana Spices,jdevereux@terana.com,+525565776678,Outbound: Instagram,Qualified,None,None,...,None,None,None,None,None,None,CR - Grocery,2022-07-27 00:00:00,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23398,00Q6T00001Ckc0RUAR,0055G0000080PMLQA2,07/20/2022,Brivino Brigaderia,viniciusalvs96@gmail.com,+55(16) 99226-9629,CR - Outbound Data Science,Qualified,Unqualified - Concept,None,...,None,None,None,None,None,None,NaN,2022-07-20 13:12:15,NaN,False
23399,00Q6T00001CmRs3UAF,005f4000005Nv3GAAS,08/14/2022,Arev Armenian cuisine - Empresa Ativa - Abre,ksulian7@gmail.com,+5511999336173,CR - Inbound Website,Qualified,None,utm_source=email&utm_medium=email&utm_channel=...,...,email,email,newuser,xxxxx,xxxxx,xxxxx,NaN,2022-08-14 16:29:50,utm_source=email&utm_channel=xxxxx&utm_campaig...,True
23401,00Q6T00001CoKB6UAN,0055G0000080RACQA2,09/09/2022,Teste,natih.borgess@gmail.com,+555511954950941,CR - Outbound Social Media,Qualified,None,None,...,None,None,None,None,None,None,NaN,2022-09-09 13:19:40,NaN,False
23402,00Q6T00001DYswOUAT,005f4000005Nv3GAAS,09/15/2022,Las Empanadas - Empresa Ativa - Abre,luis.selva@uol.com.br,+5521979207999,CR - Inbound Website,Qualified,Did Not Reach,utm_source=ig&utm_medium=paid&utm_channel=ig&u...,...,paid,ig,xxxxx,brazil_rio,VIDEO_TIKTOK-HEADLINE1-BODY1,ig,NaN,2022-09-15 23:50:11,utm_source=ig&utm_channel=ig&utm_campaign=CLOU...,True


In [21]:
## GSheets Oauth
import gspread
import pygsheets
from oauth2client.service_account import ServiceAccountCredentials
print('Init Auth')
gc = pygsheets.authorize(service_file='/home/ruben.lopez/creds_spread.json')


Init Auth


In [22]:
# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1-x_JQfzwBo_Z-81Ti2ZLPxn70k6Eyemusu8MByd2nMo')

ws = ss.worksheet('title', 'RawData')
print('clearing..')
ws.clear(start='A', end='AU')
print('pasting')
ws.set_dataframe(budget[budget['is_inbound']].reset_index(drop=True).drop(['rec_day',
                                                                           'is_inbound'], axis=1),
                 start="A1", copy_head=True, extend= True, nan='')
print('UPDATED!!','\n [PIK N PAK] Inbound Master Tracker')


Opening Spreadsheet
clearing..
pasting
UPDATED!! 
 [PIK N PAK] Inbound Master Tracker


In [23]:
#google_sheets = sheets.GoogleSheets()

#worksheet_name = "Summary"
#cell_ref = 'D1'
#google_sheets.load_spreadsheet('1-x_JQfzwBo_Z-81Ti2ZLPxn70k6Eyemusu8MByd2nMo')
#google_sheets.load_worksheet(worksheet_name)
#log_time = datetime.datetime.utcnow()
#log_time_str = log_time.strftime("%m/%d/%Y, %H:%M:%S")
#google_sheets.update_cell(cell_ref, log_time_str)

print('\nTo DeepFry File')


To DeepFry File


In [24]:
# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1W6a8o8E7lLPtjgMm-2k0HLUC3XHhNH2iiOjwL_qyaWM')

ws = ss.worksheet('title', 'RawData')
print('clearing..')
ws.clear(start='A', end='AV')
print('pasting')
ws.set_dataframe(budget.reset_index(drop=True).drop(['rec_day'], axis=1),
                 start="A1", copy_head=True, extend= True, nan='')
print('UPDATED!!',' DeepFry File')

Opening Spreadsheet
clearing..
pasting
UPDATED!!  DeepFry File


usa

In [25]:
#copia para us
print('Processing Retail Info')

budget = latam_data_no_filter[['LeadId', 'LeadOwnerId', 'LeadCreatedDate', 'Company', 'Email',
                               'Phone', 'LeadSource', 'LeadStatus', 'Main_Closed_Reason', 'Lead_Source_Detail__c',
                               'Opportunity_Stage__c', 'Estimated_Weekly_Orders__c', 'Refreshed_Lead__c',
                               'Number_of_Locations__c', 'IsWon', 'StageName', 'KitchenCity', 'KitchenCountry',
                               'UserCity', 'UserCountry', 'UserSubRegion', 'Kitchen_Type__c', 'FacilityName',
                               'QualifiedDate', 'AccountId', 'mrr', 'Negotiation_Date__c',
                               'Commitment_Date__c', 'Closed_Won_Date__c', 'General_Lost_Reason__c',
                               'Specific_Lost_Reason__c', 'TourCompleted__c', 'Final_Tour_Date__c',
                               'Facility_Tour_Date__c', 'SDR_Attribution__c', 'LeadOwnerName', 'LeadOwnerEmail',
                               'SQL_Date__c',
                               'LeadOwnerRole', 'LeadOwnerOnboardingStatus', 'RecordTypeName', 'WorkingDate',
                               'ProcessedDate', 'DaysToWorking', 'HoursToWorking', 'DaysToProcessed',
                                'UTM_Campaign__c', 'UTM_Content__c', 'UTM_Medium__c', 'UTM_Source__c', 'UTM_Term__c',
                                'utm_geo__c', 'UTM_Ad__c', 'UTM_Channel__c','Cuisine_Type__c']][latam_data_no_filter.UserRegion=='US/CAN']


budget['rec_day'] = np.where(pd.isnull(budget.Closed_Won_Date__c),
                          np.where(pd.isnull(budget.Final_Tour_Date__c),
                                   np.where(pd.isnull(budget.QualifiedDate),
                                            budget.LeadCreatedDate, budget.QualifiedDate),
                                   budget.Final_Tour_Date__c), budget.Closed_Won_Date__c)

budget['CLSD'] = budget.Lead_Source_Detail__c.apply(helpers.clean_lead_source_detail)


Processing Retail Info


In [26]:
# adding filter to 6 months
today = datetime.datetime.utcnow()
target = today - datetime.timedelta(days=180)
# budget = budget[lambda x: x['rec_day'].between(datetime.datetime(target.year, target.month, 1), today)]
print('Budget shape:', budget.shape)

budget['LeadCreatedDate'] = budget['LeadCreatedDate'].apply(lambda x: helpers.convert_date_to_str(x))
budget['QualifiedDate'] = budget['QualifiedDate'].apply(lambda x: helpers.convert_date_to_str(x))
# budget['Pitching_Date__c'] = budget['Pitching_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Negotiation_Date__c'] = budget['Negotiation_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Commitment_Date__c'] = budget['Commitment_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Closed_Won_Date__c'] = budget['Closed_Won_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Final_Tour_Date__c'] = budget['Final_Tour_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Facility_Tour_Date__c'] = budget['Facility_Tour_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['WorkingDate'] = budget['WorkingDate'].apply(lambda x: helpers.convert_date_to_str(x))
budget['ProcessedDate'] = budget['ProcessedDate'].apply(lambda x: helpers.convert_date_to_str(x))


Budget shape: (7158, 57)


In [27]:
# filter latam data
budget['is_inbound'] = budget['LeadSource'].apply(lambda x: helpers.check_is_inbound(x))



In [28]:
## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(service_file='/home/ruben.lopez/creds_spread.json')

Init Auth


In [29]:
# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('12BkLszT3CBSTLoC-H28jJYzNchlpRXwG7mMqzwxmro8')

ws = ss.worksheet('title', 'RawData')
print('clearing..')
ws.clear(start='A', end='AU')
print('pasting')
ws.set_dataframe(budget[budget['is_inbound']].reset_index(drop=True).drop(['rec_day',
                                                                           'is_inbound'], axis=1),
                 start="A1", copy_head=True, extend= True, nan='')
print('UPDATED!!')


Opening Spreadsheet
clearing..
pasting
UPDATED!!


In [30]:
#google_sheets = sheets.GoogleSheets()

#worksheet_name = "Summary"
#cell_ref = 'D1'
#google_sheets.load_spreadsheet('1-x_JQfzwBo_Z-81Ti2ZLPxn70k6Eyemusu8MByd2nMo')
#google_sheets.load_worksheet(worksheet_name)
#log_time = datetime.datetime.utcnow()
#log_time_str = log_time.strftime("%m/%d/%Y, %H:%M:%S")
#google_sheets.update_cell(cell_ref, log_time_str)


print('UPDATED!!')


UPDATED!!


In [31]:

#copia para us otro archivo
print('Processing Retail Info')

budget = latam_data_no_filter[['LeadId', 'LeadOwnerId', 'LeadCreatedDate', 'Company', 'Email',
                               'Phone', 'LeadSource', 'LeadStatus', 'Main_Closed_Reason', 'Lead_Source_Detail__c',
                               'Opportunity_Stage__c', 'Estimated_Weekly_Orders__c', 'Refreshed_Lead__c',
                               'Number_of_Locations__c', 'IsWon', 'StageName', 'KitchenCity', 'KitchenCountry',
                               'UserCity', 'UserCountry', 'UserSubRegion', 'Kitchen_Type__c', 'FacilityName',
                               'QualifiedDate', 'AccountId', 'mrr', 'Negotiation_Date__c',
                               'Commitment_Date__c', 'Closed_Won_Date__c', 'General_Lost_Reason__c',
                               'Specific_Lost_Reason__c', 'TourCompleted__c', 'Final_Tour_Date__c',
                               'Facility_Tour_Date__c', 'SDR_Attribution__c', 'LeadOwnerName', 'LeadOwnerEmail',
                               'SQL_Date__c',
                               'LeadOwnerRole', 'LeadOwnerOnboardingStatus', 'RecordTypeName', 'WorkingDate',
                               'ProcessedDate', 'DaysToWorking', 'HoursToWorking', 'DaysToProcessed',
                                'UTM_Campaign__c', 'UTM_Content__c', 'UTM_Medium__c', 'UTM_Source__c', 'UTM_Term__c',
                                'utm_geo__c','UTM_Ad__c', 'UTM_Channel__c' ]][latam_data_no_filter.UserRegion=='US/CAN']


budget['rec_day'] = np.where(pd.isnull(budget.Closed_Won_Date__c),
                          np.where(pd.isnull(budget.Final_Tour_Date__c),
                                   np.where(pd.isnull(budget.QualifiedDate),
                                            budget.LeadCreatedDate, budget.QualifiedDate),
                                   budget.Final_Tour_Date__c), budget.Closed_Won_Date__c)

budget['CLSD'] = budget.Lead_Source_Detail__c.apply(helpers.clean_lead_source_detail)


Processing Retail Info


In [32]:
# adding filter to 6 months
today = datetime.datetime.utcnow()
target = today - datetime.timedelta(days=180)
# budget = budget[lambda x: x['rec_day'].between(datetime.datetime(target.year, target.month, 1), today)]
print('Budget shape:', budget.shape)

Budget shape: (7158, 56)


In [33]:
budget['LeadCreatedDate'] = budget['LeadCreatedDate'].apply(lambda x: helpers.convert_date_to_str(x))
budget['QualifiedDate'] = budget['QualifiedDate'].apply(lambda x: helpers.convert_date_to_str(x))
# budget['Pitching_Date__c'] = budget['Pitching_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Negotiation_Date__c'] = budget['Negotiation_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Commitment_Date__c'] = budget['Commitment_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Closed_Won_Date__c'] = budget['Closed_Won_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Final_Tour_Date__c'] = budget['Final_Tour_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['Facility_Tour_Date__c'] = budget['Facility_Tour_Date__c'].apply(lambda x: helpers.convert_date_to_str(x))
budget['WorkingDate'] = budget['WorkingDate'].apply(lambda x: helpers.convert_date_to_str(x))
budget['ProcessedDate'] = budget['ProcessedDate'].apply(lambda x: helpers.convert_date_to_str(x))

In [34]:
# filter latam data
budget['is_inbound'] = budget['LeadSource'].apply(lambda x: helpers.check_is_inbound(x))


In [35]:
## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(service_file='/home/ruben.lopez/creds_spread.json')

Init Auth


In [36]:
# open the spreedsheet second file
print('Opening Spreadsheet second file')

#ss = gc.open_by_key('1JLVfXaDBgWyvzZ64z2TS314cgPZQx2Zw25crVuvq5XM')

#ws = ss.worksheet('title', 'RawData')
print('clearing..')
#ws.clear(start='A', end='AU')
print('pasting')
#ws.set_dataframe(budget,start="A1", copy_head=True, extend= True, nan='')
print('UPDATED!! second file')
print('UPDATED!! all')

Opening Spreadsheet second file
clearing..
pasting
UPDATED!! second file
UPDATED!! all
